In [2]:
%pylab inline

import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


C:\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [13]:
df=pd.read_csv("train.csv")
train=df[:1001]
test=df[1001:]
train_x=train[train.columns[1:5]].values
train_y=train[train.columns[5]].values.reshape(len(train),1)
test_x=test[test.columns[1:5]].values
test_y=test[test.columns[5]].values.reshape(len(test),1)


In [17]:
train_x,train_y=shuffle(train_x,train_y,random_state=1)
test_x,test_y=shuffle(test_x,test_y,random_state=1)


In [5]:
'''#Reading the dataset
def read_dataset():
    df=pd.read_csv('train.csv')
    X=df[df.columns[1:5]].values
    y=df[df.columns[5]]
    
    #Encode the dependent variable
    encoder=LabelEncoder()
    encoder.fit(y)
    y=encoder.transform(y)
    Y=one_hot_encode(y)
    print(X.shape)
    return (X,Y)'''

In [6]:
#define the encoder function
'''def one_hot_encode(labels):
    n_labels=len(labels)
    n_unique_labels=len(np.unique(labels))
    one_hot_encode=np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels),labels]=1
    return one_hot_encode'''

In [25]:
X,Y=read_dataset()
X,Y=shuffle(X,Y,random_state=1)


(1060, 4)


NameError: name 'df' is not defined

In [18]:
#train_x,test_x,train_y,test_y=train_test_split(X,Y,test_size=0.20)
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
train_y

(1001, 4)
(1001, 1)
(59, 4)


array([[0],
       [0],
       [1],
       ...,
       [0],
       [0],
       [1]], dtype=int64)

In [28]:
learning_rate=0.01
epochs=1000
cost_history=np.empty(shape=[1],dtype=float)
n_dim=train.shape[1]
n_class=1


In [29]:
n_hidden_1=10
n_hidden_2=10
n_hidden_3=10
n_hidden_4=10

In [30]:
x=tf.placeholder(tf.float32,[None,n_dim])
w=tf.Variable(tf.zeros([n_dim,n_class]))
b=tf.Variable(tf.zeros([n_dim]))
y_=tf.placeholder(tf.float32,[None,n_class])

In [31]:
weights ={
          'h1':tf.Variable(tf.truncated_normal([n_dim, n_hidden_1])),
          'h2':tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2])),
          'h3':tf.Variable(tf.truncated_normal([n_hidden_2, n_hidden_3])),
          'h4':tf.Variable(tf.truncated_normal([n_hidden_3, n_hidden_4])),
         'out':tf.Variable(tf.truncated_normal([n_hidden_4,n_class]))
        }
biases ={
          'b1':tf.Variable(tf.truncated_normal([n_hidden_1])),
          'b2':tf.Variable(tf.truncated_normal([n_hidden_2])),
          'b3':tf.Variable(tf.truncated_normal([n_hidden_3])),
          'b4':tf.Variable(tf.truncated_normal([n_hidden_4])),
         'out':tf.Variable(tf.truncated_normal([n_class]))
}

In [32]:
def neural_network(x,weights,biases):
    layer_1=tf.add(tf.matmul(x,weights['h1']),biases['b1'])
    layer_1=tf.nn.sigmoid(layer_1)
    
    layer_2=tf.add(tf.matmul(layer_1,weights['h2']),biases['b2'])
    layer_2=tf.nn.sigmoid(layer_2)
    
    layer_3=tf.add(tf.matmul(layer_2,weights['h3']),biases['b3'])
    layer_3=tf.nn.sigmoid(layer_3)
    
    layer_4=tf.add(tf.matmul(layer_3,weights['h4']),biases['b4'])
    layer_4=tf.nn.relu(layer_4)
    
    out_layer=tf.matmul(layer_4,weights['out'])+biases['out']
    return out_layer

    

In [33]:
init=tf.global_variables_initializer()
saver=tf.train.Saver()

In [34]:
y=neural_network(x,weights,biases)

cost_function=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y,labels=y_))
training_step=tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)
sess=tf.Session()
sess.run(init)

In [35]:
mse_history=[]
accuracy_history=[]
for epoch in range(epochs):
    sess.run(training_step,feed_dict={x:train_x,y_:train_y})
    cost=sess.run(cost_function,feed_dict={x:train_x,y_:train_y})
    cost_history=np.append(cost_history,cost)
    correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
    accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    
    pred_y=sess.run(y,feed_dict={x:test_x})
    mse=tf.reduce_mean(tf.square(pred_y - test_y))
    mse_=sess.run(mse)
    mse_history.append(mse_)
    accuracy=(sess.run(accuracy,feed_dict={x:train_x,y_:train_y}))
    accuracy_history.append(accuracy)
    print('epoch: ',epoch," - ","cost :", cost," -MSE ",mse_," -Training accuracy :",accuracy)
    
plt.plot(accuracy_history)
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.show()

ValueError: Cannot feed value of shape (1001, 4) for Tensor 'Placeholder_2:0', which has shape '(?, 6)'

In [20]:
#Print the final accuracy
correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
print("Test Accuracy: ",(sess.run(accuracy, feed_dict={x:test_x,y_:test_y})))

pred_y=sess.run(y, feed_dict={x:test_x})
mse=tf.reduce_mean(tf.square(pred_y-test_y))
print("MSE: %.4f" %sess.run(mse))

Test Accuracy:  0.8443396
MSE: 9.0531


In [24]:
test_y

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.